In [17]:
Alpha_Name = "AlphaNet_Original_Input_5d_return"
start_time = "2020-06-01"
forecast_months = 6 # months
feat_num = 9
sequence = 3

In [18]:
import os
from os import walk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
from torch.autograd import Variable
from progressbar import ProgressBar
from tqdm import tqdm
import time
import multiprocessing as mp

path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
output_path = "/home/wuwenjun/Alpha_Factor/" + "LSTM_" + Alpha_Name + "/result/"
model_dir = "/home/wuwenjun/Alpha_Factor/" + "LSTM_" + Alpha_Name + "/model/"
input_length = feat_num * (feat_num - 1) + 4 * feat_num
if os.path.exists(output_path) == False:
    os.makedirs(output_path)
    print(output_path)

In [19]:
time_list = []
data_path = path + "Final/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        time_list.append(j)
time_list.sort()     
for count,item in enumerate(time_list):
    if item.startswith(start_time):
        train_timestamp = time_list[:count]
        test_timestamp = time_list[count:count + forecast_months//3]
        break
display(train_timestamp)
display(test_timestamp)

model_path = (model_dir + test_timestamp[0].split("_")[0] + "_" + test_timestamp[-1].split("_")[-1]).replace(".par","/")
if os.path.exists(model_path) == False:
    os.makedirs(model_path)
    print(model_path)

100%|██████████| 26/26 [00:00<00:00, 190650.18it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par']

['2020-06-01_2020-09-01.par', '2020-09-01_2021-01-01.par']

In [20]:
trainx , trainy , testx , testy = [] , [] , [],  []

for train in tqdm(train_timestamp):
    df = pd.read_parquet(path+ "Final/" + train).set_index(["timestamp","ticker"])
    trainx.append(df.drop("target",axis=1))
    trainy.append(df['target'])
trainx = pd.concat(trainx,axis=0)
trainy = pd.concat(trainy,axis=0)

display(trainx)
display(trainy)


for test in tqdm(test_timestamp):
    df = pd.read_parquet(path+ "Final/" + test).set_index(["timestamp","ticker"])
    testx.append(df.drop("target",axis=1))
    testy.append(df['target'])
testx = pd.concat(testx,axis=0)
testy = pd.concat(testy,axis=0)
target_list = pd.DataFrame(testy.copy())
display(testx)
display(testy)

100%|██████████| 22/22 [00:31<00:00,  1.43s/it]


,,0,1,2,3,4,5,6,7,8,9,...,314,315,316,317,318,319,320,321,322,323
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,1,1047.509016,165.116922,249.851660,924.464829,243.390500,183.194222,926.891342,158.088300,78.887819,957.587687,...,861850.412364,0.138453,-0.376716,0.284385,1.522075,1.245018,0.876160,3.017927,2.468587,1.737222
2015-02-17 09:30:00,1,1284.106922,304.746632,233.319300,1251.132811,325.347890,190.544389,1311.161194,268.878420,78.529422,1262.525551,...,825372.445636,-1.631951,-0.601635,0.240629,1.601496,1.147820,0.839082,3.175407,2.275878,1.663689
2015-02-25 09:30:00,1,1515.538580,320.840611,242.242430,1522.025569,325.437271,214.401980,1539.154187,259.118036,99.405562,1482.690747,...,784033.056545,-1.377793,-0.151133,0.292173,1.574869,1.078356,0.797058,3.122595,2.138158,1.580369
2015-02-26 09:30:00,1,1466.712161,355.378729,244.332118,1546.573639,318.650776,222.730596,1381.349806,297.507667,108.085628,1469.916341,...,727636.666545,-0.375218,-0.482991,0.038896,1.643798,1.006125,0.739729,3.259280,1.994944,1.466684
2015-02-27 09:30:00,1,1271.480107,335.406489,121.012389,1338.104279,252.973056,186.957378,1296.812044,215.993378,108.132489,1324.356569,...,756908.456182,-0.411738,-0.287113,0.381687,1.581575,1.159289,0.769491,3.135918,2.298631,1.525691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-25 09:30:00,688399,145.058544,102.900373,291.804320,96.556694,98.291493,310.663984,135.863911,83.149802,208.860347,115.710131,...,29397.982364,5.979449,1.024996,-0.270202,16.970444,19.273864,22.039995,16.970444,19.273864,22.039995
2020-05-26 09:30:00,688399,188.140111,73.147256,141.024870,137.441784,69.128806,153.068290,163.341413,51.128161,98.400247,157.379208,...,27335.442182,4.343320,3.362193,-0.479024,17.499125,19.485409,20.493689,17.499125,19.485409,20.493689
2020-05-27 09:30:00,688399,250.251258,116.622329,92.636120,199.366691,104.925807,100.881953,236.665444,97.351904,42.759193,223.721218,...,25205.631636,6.939820,3.436735,-0.352345,19.836240,20.228593,18.896940,19.836240,20.228593,18.896940


timestamp            ticker
2015-02-16 09:30:00  1        -0.679514
2015-02-17 09:30:00  1        -0.824485
2015-02-25 09:30:00  1        -0.790990
2015-02-26 09:30:00  1        -1.006040
2015-02-27 09:30:00  1        -1.213373
                                 ...   
2020-05-25 09:30:00  688399    0.702991
2020-05-26 09:30:00  688399    0.940820
2020-05-27 09:30:00  688399    0.229983
2020-05-28 09:30:00  688399   -0.946442
2020-05-29 09:30:00  688399   -0.852920
Name: target, Length: 4161372, dtype: float64

100%|██████████| 2/2 [00:04<00:00,  2.01s/it]


,,0,1,2,3,4,5,6,7,8,9,...,314,315,316,317,318,319,320,321,322,323
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2020-06-01 09:30:00,1,882.944203,477.380537,390.200912,842.725349,568.429177,354.534178,839.401301,399.829017,295.774366,870.477016,...,594983.158182,0.597235,-0.700805,0.058007,0.482675,0.392376,0.306609,1.088991,0.885267,0.691756
2020-06-02 09:30:00,1,1079.326667,915.056551,427.328343,940.314889,835.153424,384.294353,1070.338571,842.106288,340.038796,1063.258583,...,645614.308000,1.176427,-0.705253,0.504360,0.495927,0.376684,0.332698,1.118900,0.849836,0.750629
2020-06-03 09:30:00,1,1572.699599,790.752224,592.441519,1378.253330,750.328317,496.234341,1301.360267,792.500389,513.409983,1484.646087,...,691867.845455,0.900685,-0.376718,0.768235,0.477851,0.361953,0.356540,1.078122,0.816591,0.804396
2020-06-04 09:30:00,1,1326.512511,798.442430,1125.481122,1129.034622,786.597770,950.670667,1059.202761,781.086124,874.377233,1204.649875,...,746309.596727,0.517645,-0.098129,0.695389,0.477236,0.357747,0.384602,1.076756,0.807105,0.867689
2020-06-05 09:30:00,1,898.336100,800.632859,1346.137542,811.436767,757.376037,1178.652987,721.533522,765.874296,1146.245882,818.837005,...,727963.720545,0.288802,-0.213020,0.698009,0.447205,0.343540,0.375153,1.009002,0.775055,0.846360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-25 09:30:00,688981,0.376847,0.558021,7.531216,0.289600,0.857656,8.524438,0.415018,0.554211,7.358793,0.274589,...,346090.446727,-0.797787,0.426938,-0.979651,2.151998,3.405911,3.327036,2.151998,3.405911,3.327036
2020-12-28 09:30:00,688981,0.875378,0.480857,6.125189,0.763667,0.835368,6.400609,0.808300,0.502249,5.499779,0.747445,...,315999.365091,-0.612075,0.348475,-0.959076,2.035465,3.520044,3.037771,2.035465,3.520044,3.037771
2020-12-29 09:30:00,688981,0.713691,0.488609,3.956496,0.914182,0.635866,3.450882,0.796116,0.431834,3.023371,0.773336,...,296250.213636,-0.408900,0.171973,-1.200438,2.207042,3.282189,2.847920,2.207042,3.282189,2.847920


timestamp            ticker
2020-06-01 09:30:00  1         0.151141
2020-06-02 09:30:00  1         0.207991
2020-06-03 09:30:00  1        -0.349630
2020-06-04 09:30:00  1        -0.315796
2020-06-05 09:30:00  1        -0.488418
                                 ...   
2020-12-25 09:30:00  688981    0.838607
2020-12-28 09:30:00  688981    1.083800
2020-12-29 09:30:00  688981    1.815341
2020-12-30 09:30:00  688981    1.966492
2020-12-31 09:30:00  688981    0.706211
Name: target, Length: 572224, dtype: float64

In [21]:
# Reshape
trainx = torch.from_numpy(np.array(trainx).reshape(-1,input_length,sequence))
trainy = torch.from_numpy(np.array(trainy).reshape(-1,1))
testx = torch.from_numpy(np.array(testx).reshape(-1,input_length,sequence))
testy = torch.from_numpy(np.array(testy).reshape(-1,1))
print("trainx.shape: " , trainx.shape)
print("trainy.shape: " , trainy.shape)
print("testx.shape: " , testx.shape)
print("testy.shape: " , testy.shape)

trainx.shape:  torch.Size([4161372, 108, 3])
trainy.shape:  torch.Size([4161372, 1])
testx.shape:  torch.Size([572224, 108, 3])
testy.shape:  torch.Size([572224, 1])


In [22]:
train_dataset = Data.TensorDataset(trainx, trainy)
test_dataset = Data.TensorDataset(testx, testy)
batch_size = 1024
train_loader = Data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=16,
    pin_memory=True
)

test_loader = Data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=16,
    pin_memory=True
)

In [23]:
class AlphaNet_LSTM(nn.Module):
    def __init__(self, factor_num, fully_connect_layer_neural):
        super(AlphaNet_LSTM, self).__init__()
        self.fc1_neuron = factor_num # 108
        self.fc2_neuron = fully_connect_layer_neural # 30
        
        # Layer
        self.batch = torch.nn.BatchNorm1d(self.fc1_neuron)
        self.lstm = nn.LSTM(self.fc1_neuron,self.fc2_neuron,2,batch_first = True,bidirectional = True)
        self.batch2 = torch.nn.BatchNorm1d(self.fc2_neuron)
        self.dropout = nn.Dropout(0.3)
#         self.relu = nn.ReLU()
        self.out = nn.Linear(self.fc2_neuron, 1)

    def forward(self, x):
        x = self.batch(x)
        x = torch.transpose(x,1,2)
        _,(hn, cn) = self.lstm(x) # hn.shape: torch.Size([4, 512, 30])
        hn = hn[-1,:,:] # torch.Size([512, 30])
        hn = self.batch2(hn) # torch.Size([512, 30])
#         hn = self.relu(hn)
        hn = self.dropout(hn)
        y_pred = self.out(hn)
        return y_pred
# device=torch.device('cuda:0')

In [ ]:
alphanet = AlphaNet_LSTM(input_length, 30)
alphanet = alphanet.cuda()
# alphanet = torch.nn.parallel.DataParallel(alphanet)
print(alphanet)
total_length = trainx.shape[0]
LR = 0.01
loss_function = nn.MSELoss().cuda()
optimizer = optim.Adam(alphanet.parameters(), lr=LR)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=2,gamma = 0.5)
epoch_num = 50
loss_list = []

min_loss = float("inf")
for epoch in tqdm(range(epoch_num)):
    total_loss = 0
    for _, (inputs, outputs) in enumerate(train_loader):
        inputs = Variable(inputs).float().cuda()
        outputs = Variable(outputs).float().cuda()
        optimizer.zero_grad() # noticed:  the grad return to zero before starting the loop
        
        # forward + backward +update
        pred = alphanet(inputs)
        pred = pred.cuda()
        loss = loss_function(pred, outputs)
        loss.backward()
        optimizer.step()
        
#         lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
        total_loss += loss.item()
    total_loss = total_loss * batch_size / total_length
    print('Epoch: ', epoch + 1, ' loss: ', total_loss)
    loss_list.append(total_loss)
    if total_loss < min_loss:
        torch.save(alphanet,model_path + "best_model.tar")
    np.save(model_path + "loss.npy", loss_list)
torch.save(alphanet,model_path + "model.tar")

  0%|          | 0/50 [00:00<?, ?it/s]

AlphaNet_LSTM(
  (batch): BatchNorm1d(108, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(108, 30, num_layers=2, batch_first=True, bidirectional=True)
  (batch2): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (out): Linear(in_features=30, out_features=1, bias=True)
)


  2%|▏         | 1/50 [01:37<1:19:55, 97.87s/it]

Epoch:  1  loss:  0.8268904241538824


  4%|▍         | 2/50 [03:19<1:20:05, 100.12s/it]

Epoch:  2  loss:  0.822135600908599


  6%|▌         | 3/50 [05:28<1:28:54, 113.50s/it]

Epoch:  3  loss:  0.8207321919046471


In [ ]:
alphanet = torch.load(model_path + "best_model.tar")
alphanet = alphanet.cuda()
# alphanet = torch.nn.parallel.DataParallel(alphanet)
print(alphanet)
total_length = trainx.shape[0]
loss_function = nn.MSELoss().cuda()
optimizer = optim.Adam(alphanet.parameters(), lr=0.001)
loss_list = list(np.load(model_path + "loss.npy"))
min_loss = min(loss_list)
for epoch in tqdm(range(50)):
    total_loss = 0
    for _, (inputs, outputs) in enumerate(train_loader):
        inputs = Variable(inputs).float().cuda()
        outputs = Variable(outputs).float().cuda()
        optimizer.zero_grad() # noticed:  the grad return to zero before starting the loop
        
        # forward + backward +update
        pred = alphanet(inputs)
        pred = pred.cuda()
        loss = loss_function(pred, outputs)
        loss.backward()
        optimizer.step()
        
#         lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
        total_loss += loss.item()
    total_loss = total_loss * batch_size / total_length
    print('Epoch: ', epoch + 1, ' loss: ', total_loss)
    loss_list.append(total_loss)
    if total_loss < min_loss:
        torch.save(alphanet,model_path + "best_model.tar")
    np.save(model_path + "loss.npy", loss_list)
torch.save(alphanet,model_path + "model.tar")

In [ ]:
plt.plot(loss_list,color = 'r')
plt.savefig(model_path+"loss.png")

In [ ]:
alphanet = alphanet.cpu()
pred_list = []
label_list = []
for _, (data, label) in enumerate(test_loader):
    data = Variable(data).float()
    pred = alphanet(data)
    pred_list.extend(pred.tolist())
    label_list.extend(label.tolist())

In [ ]:
final = pd.DataFrame(pred_list)
final = pd.concat([target_list.reset_index(),final],axis=1)
final.rename(columns={0:Alpha_Name,'ticker': 'symbol'},inplace=True)
# final.set_index(["timestamp","symbol"],inplace=True)
final

In [ ]:
final.to_parquet(output_path
                + test_timestamp[0].split("_")[0] + "_" + test_timestamp[-1].split("_")[-1])